In [ ]:
# %matplotlib ipympl
%matplotlib inline
import sys
from pathlib import Path

import hyperspy.api as hs
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from matplotlib import gridspec, patches, ticker, transforms
from matplotlib.colorbar import Colorbar
from matplotlib.colors import ListedColormap


In [ ]:
# Ensure custom module path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# 颜色设定
colors = list(tol_cset("vibrant"))
if r"sunset" not in plt.colormaps():
    plt.colormaps.register(tol_cmap("sunset"))
if r"rainbow_PuRd" not in plt.colormaps():
    plt.colormaps.register(tol_cmap("rainbow_PuRd"))  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

In [ ]:
# 读取数据
path_xps = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XPS+UPS\ExSitu\αMnO2\Pristine\Powder\Results\2023-ICN2\V1"
)
Mn_xps = pd.read_csv(
    Path.joinpath(path_xps, r"Mn2p3_2.dat"), sep=r"\s+", skiprows=2, index_col=None, header=1, comment="#"
)
K_xps = pd.read_csv(Path.joinpath(path_xps, "K2p.dat"), sep=r"\s+", skiprows=2, index_col=None, header=1, comment="#")
O_xps = pd.read_csv(Path.joinpath(path_xps, "O1s.dat"), sep=r"\s+", skiprows=2, index_col=None, header=1, comment="#")

In [ ]:
# 数据处理
# Mn
envelope_Mn_IV = Mn_xps.iloc[:, 3:11].sum(axis=1) - (Mn_xps.iloc[:, -4] * 7)  # envelope_Mn_IV
envelope_Mn_III = Mn_xps.iloc[:, 11:19].sum(axis=1) - (Mn_xps.iloc[:, -4] * 7)  # envelope_Mn_III
envelope_Mn_II = Mn_xps.iloc[:, 19:27].sum(axis=1) - (Mn_xps.iloc[:, -4] * 7)  # envelope_Mn_II
# K
if r"Background_K_2p" not in K_xps.columns:
    K_xps.insert(11, "Background_K_2p", K_xps.iloc[:, 7].copy(deep=True))

for i in range(K_xps.shape[1] - 5):
    K_xps.iloc[:, i + 2] -= K_xps.loc[:, "Background_K_2p"]
# K_xps.info()
# O
if r"Background_O_1s" not in O_xps.columns:
    O_xps.insert(10, "Background_O_1s", O_xps.iloc[:, 6].copy(deep=True))

for i in range(O_xps.shape[1] - 5):
    O_xps.iloc[:, i + 2] -= O_xps.loc[:, "Background_O_1s"]
# O_xps.info()

In [ ]:
# Mn
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig_e = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig_e.add_axes((0, 0, 1, 1), zorder=0)
ax.plot(Mn_xps.iloc[:, 1], Mn_xps.iloc[:, 2], label=r"Data", color=colors[0])  # each peak of Mn_xps_data
ax.plot(
    Mn_xps.iloc[:, 1],
    Mn_xps.iloc[:, 28],
    label=r"Fit",
    color=colors[1],
    linestyle="--",
)  # envelope_all
ax.plot(Mn_xps.iloc[:, 1], Mn_xps.iloc[:, -4], label=r"Background", color=colors[2], zorder=3)  # background

ax.plot(Mn_xps.iloc[:, 1], envelope_Mn_IV, color=colors[3], linestyle="--", label=r"Mn(IV)")  # envelope_Mn_IV
ax.plot(Mn_xps.iloc[:, 1], envelope_Mn_III, color=colors[5], linestyle="--", label=r"Mn(III)")  # envelope_Mn_III
ax.plot(Mn_xps.iloc[:, 1], envelope_Mn_II, color=colors[4], linestyle="--", label=r"Mn(II)", zorder=3)  # envelope_Mn_II

ax.plot(Mn_xps.iloc[:, 1], Mn_xps.iloc[:, 3:11], color=colors[3], alpha=0.5)  # each peak of Mn_xps_Mn_IV
for i in range(8):
    ax.fill_between(
        x=Mn_xps.iloc[:, 1], y1=Mn_xps.iloc[:, i + 3], y2=Mn_xps.iloc[:, -4], facecolor=colors[3], alpha=0.3
    )
ax.plot(Mn_xps.iloc[:, 1], Mn_xps.iloc[:, 11:19], color=colors[5], alpha=0.5)  # each peak of Mn_xps_Mn_III
for i in range(8):
    ax.fill_between(
        x=Mn_xps.iloc[:, 1], y1=Mn_xps.iloc[:, i + 11], y2=Mn_xps.iloc[:, -4], facecolor=colors[5], alpha=0.3
    )
ax.plot(Mn_xps.iloc[:, 1], Mn_xps.iloc[:, 19:27], color=colors[4], alpha=0.5)  # each peak of Mn_xps_Mn_II
for i in range(8):
    ax.fill_between(
        x=Mn_xps.iloc[:, 1], y1=Mn_xps.iloc[:, i + 19], y2=Mn_xps.iloc[:, -4], facecolor=colors[4], alpha=0.3
    )

ax.legend(
    loc="upper left",
    bbox_to_anchor=(0, 1.02),
    ncols=2,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.8,
    handletextpad=0.5,
)

ax.text(
    0.03,
    0.65,
    r"$\alpha$MnO${_2}:$ 3.45(5)",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[0],
)
ax.text(
    0.03,
    0.55,
    r"Mn(III): 48.8(3) at.%",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[5],
)
ax.text(
    0.03,
    0.45,
    r"Mn(IV): 49.7(6) at.%",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=9,
    color=colors[3],
)

ax.text(
    0.95,
    0.95,
    r"Mn $\mathit{2p_{3/2}}$",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=12,
)

ax.set_xlabel(
    r"Bind Energy (eV)",
    fontsize=11,
)
ax.set_xlim(648.5, 638.5)
ax.xaxis.set_minor_locator(ticker.LinearLocator(21))

ax.set_ylabel(
    r"Intensity (counts)",
    fontsize=11,
    labelpad=0,
)
ax.set_ylim(2000, 15000)
ax.yaxis.set_major_formatter(ticker.EngFormatter(sep=" "))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1000))
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)

plt.savefig(
    Path.joinpath(path_out, r"XPS_Mn_2p_300.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.savefig(
    Path.joinpath(path_out, r"XPS_Mn_2p_600.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
# O
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig_a = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig_a.add_axes((0, 0, 1.0, 1.0), zorder=0)

ax.plot(O_xps.iloc[:, 1], O_xps.iloc[:, 2], ls="-", lw=1.0, c=colors[0], label="Data")
ax.plot(O_xps.iloc[:, 1], O_xps.iloc[:, 7], ls="--", lw=1.0, c=colors[1], label="Fit")
ax.plot(O_xps.iloc[:, 1], O_xps.iloc[:, 6], ls="-", lw=1.0, c=colors[2], label="Background", zorder=6)

ax.plot(O_xps.iloc[:, 1], O_xps.iloc[:, 3], ls="-", lw=1.0, c=colors[3], label=r"Lattice Oxide")
ax.fill_between(x=O_xps.iloc[:, 1], y1=O_xps.iloc[:, 3], y2=O_xps.iloc[:, 6], facecolor=colors[3], alpha=0.5)
ax.plot(
    O_xps.iloc[:, 1], O_xps.iloc[:, 4], ls="-", lw=1.0, c=colors[4], label="Hydroxide, Hydrated\n or Defective Oxide"
)
ax.fill_between(x=O_xps.iloc[:, 1], y1=O_xps.iloc[:, 4], y2=O_xps.iloc[:, 6], facecolor=colors[4], alpha=0.5)
ax.plot(O_xps.iloc[:, 1], O_xps.iloc[:, 5], ls="-", lw=1.0, c=colors[5], label=r"Water")
ax.fill_between(x=O_xps.iloc[:, 1], y1=O_xps.iloc[:, 5], y2=O_xps.iloc[:, 6], facecolor=colors[5], alpha=0.5)

ax.legend(loc="upper left", bbox_to_anchor=(0, 1), ncols=1, frameon=False, labelcolor="linecolor", fontsize=8)
ax.set_xlim(535.5, 526.5)
ax.set_ylim(0, 14000)
ax.set_xlabel(
    "Bind Energy (eV)",
    fontsize=11,
)
ax.set_ylabel(
    "Intensity (counts)",
    fontsize=11,
)

ax.yaxis.set_major_formatter(ticker.EngFormatter(sep=" "))
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3000))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1500))
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)

ax.annotate(
    r"529.5(7) eV",
    (0.65, 0.85),
    xytext=(0.72, 0.6),
    fontsize=9,
    fontweight="bold",
    xycoords="axes fraction",
    textcoords="axes fraction",
    c=colors[3],
    arrowprops={"arrowstyle": patches.ArrowStyle("->"), "color": colors[3], "ls": "--"},
)
ax.annotate(
    r"530.8(3) eV",
    (0.53, 0.16),
    xytext=(0.25, 0.28),
    fontsize=9,
    fontweight="bold",
    xycoords="axes fraction",
    textcoords="axes fraction",
    c=colors[4],
    arrowprops={"arrowstyle": patches.ArrowStyle("->"), "color": colors[4], "ls": "--"},
)
ax.annotate(
    r"532.8(8) eV",
    (0.25, 0.02),
    xytext=(0.1, 0.17),
    fontsize=9,
    c=colors[5],
    xycoords="axes fraction",
    textcoords="axes fraction",
    fontweight="bold",
    arrowprops={"arrowstyle": patches.ArrowStyle("->"), "color": colors[5], "ls": "--"},
)
ax.text(
    0.95,
    0.95,
    r"O $\mathit{1s}$",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=12,
)

plt.savefig(
    Path.joinpath(path_out, r"XPS_O_1s_300.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.savefig(
    Path.joinpath(path_out, r"XPS_O_1s_600.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
# K
# gridspec inside gridspec
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig_b = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig_b.add_axes((0, 0, 1.0, 1.0), zorder=0)

ax.plot(K_xps.iloc[:, 1], K_xps.iloc[:, 2], ls="-", lw=1.0, c=colors[0], label="Data")
ax.plot(K_xps.iloc[:, 1], K_xps.iloc[:, 8], ls="--", lw=1.0, c=colors[1], label="Fit")
ax.plot(K_xps.iloc[:, 1], K_xps.iloc[:, 7], ls="-", lw=1.0, c=colors[2], label="Background", zorder=6)

ax.plot(K_xps.iloc[:, 1], K_xps.iloc[:, 3:5], ls="-", lw=1.0, c=colors[3], label=[r"K-Mn", None])
ax.fill_between(
    x=K_xps.iloc[:, 1], y1=K_xps.iloc[:, 3], y2=K_xps.iloc[:, 7], facecolor=colors[3], alpha=0.5, label=None
)
ax.fill_between(
    x=K_xps.iloc[:, 1], y1=K_xps.iloc[:, 4], y2=K_xps.iloc[:, 7], facecolor=colors[3], alpha=0.5, label=None
)
ax.plot(K_xps.iloc[:, 1], K_xps.iloc[:, 5:7], ls="-", lw=1.0, c=colors[4], label=[r"K-O", None])
ax.fill_between(
    x=K_xps.iloc[:, 1], y1=K_xps.iloc[:, 5], y2=K_xps.iloc[:, 7], facecolor=colors[4], alpha=0.5, label=None
)
ax.fill_between(
    x=K_xps.iloc[:, 1], y1=K_xps.iloc[:, 6], y2=K_xps.iloc[:, 7], facecolor=colors[4], alpha=0.5, label=None
)

ax.legend(loc="upper left", bbox_to_anchor=(0, 1), ncols=2, frameon=False, labelcolor="linecolor", fontsize=8)
ax.set_xlim(296.5, 290.3)
ax.set_xlabel(
    "Bind Energy (eV)",
    fontsize=11,
)
ax.set_ylim(0, 2400)
ax.set_ylabel(
    "Intensity (counts)",
    fontsize=11,
)
ax.tick_params(axis="x", labelsize=9)
ax.tick_params(axis="y", labelsize=9)
ax.yaxis.set_major_formatter(ticker.EngFormatter(sep=" "))
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=600))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=300))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))

ax.text(
    0.98,
    0.95,
    r"K $\mathit{2p}$",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    fontsize=12,
)

ax.text(
    0.97,
    0.6,
    r"K 2p$\mathit{\mathrm{_{3/2}}}$",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    color=colors[5],
    fontsize=10,
)
ax.text(
    0.22,
    0.4,
    r"K 2p$\mathit{\mathrm{_{1/2}}}$",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    color=colors[5],
    fontsize=10,
)

ax.axvline(x=291.82, color="black", ymin=0.67, ymax=0.8, linestyle="--")
ax.axvline(x=294.58, color="black", ymin=0.35, ymax=0.7, linestyle="--")
ax.text(
    0.63,
    0.65,
    r"2.7(7) eV",
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
    weight="bold",
    fontsize=10,
    c="black",
)
ax.annotate(
    "",
    xy=(0.76, 0.69),
    xytext=(0.305, 0.69),
    xycoords="axes fraction",
    textcoords="axes fraction",
    arrowprops={"arrowstyle": "<->", "color": "black", "lw": 1},
    zorder=10,
)

plt.savefig(
    Path.joinpath(path_out, r"XPS_K_2p_300.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)

plt.savefig(
    Path.joinpath(path_out, r"XPS_K_2p_600.tif"),
    pad_inches=0.1,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()